In [2]:
import comet_ml
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling
import datasets
from transformers import logging, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from itertools import combinations
import numpy as np
import api_keys
import re

In [3]:
experiment = comet_ml.Experiment(api_key=api_keys.comet, project_name='rap-lyrics-generator-llm')

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/artaasd95/rap-lyrics-generator-llm/2087a985394e472a8284a1932c4dfe95



COMET WARNING: Unknown error exporting current conda environment
COMET WARNING: Unknown error retrieving Conda package as an explicit file
COMET WARNING: Unknown error retrieving Conda information


In [3]:
experiment.log_parameters({
    "num_train_epochs": 3,
    "per_device_train_batch_size": 2,
    # Add any other relevant hyperparameters here
})

In [4]:
rap_lyrics_train_dataset = datasets.load_dataset("nateraw/rap-lyrics-v2", split='train')
#rap_lyrics_train_dataset = rap_lyrics_train_dataset[:int(len(rap_lyrics_train_dataset)*0.7)]
#rap_lyrics_test_dataset = datasets.load_dataset("nateraw/rap-lyrics-v2", split='train')[int(len(rap_lyrics_train_dataset)*0.7):]

Using the latest cached version of the dataset since nateraw/rap-lyrics-v2 couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at C:\Users\arta\.cache\huggingface\datasets\nateraw___rap-lyrics-v2\default\0.0.0\21c94ae389238becad55271b69426ac99168db1e (last modified on Mon Nov  4 10:05:56 2024).


In [5]:
model_name = "gpt2"  # You could use a larger model like gpt2-medium for better performance
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


In [6]:
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=8092)

# Tokenize dataset
tokenized_train_dataset = rap_lyrics_train_dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Create a data collator for dynamic batching
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# tokenized_test_dataset = rap_lyrics_train_dataset.map(tokenize_function, batched=True, remove_columns=["text"])
# data_collator_test = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

AttributeError: 'dict' object has no attribute 'map'

In [ ]:
# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=2,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir='./logs',
    logging_steps=100,
    report_to="comet_ml",  # Ensure training logs are sent to Comet
)

# Instantiate Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_train_dataset,
    #eval_dataset=tokenized_test_dataset,
    compute_metrics=None,  # Add compute_metrics function if needed
)

# Fine-tune the model
trainer.train()

In [ ]:
# Save the fine-tuned model
model.save_pretrained('./fine_tuned_rap_model')
tokenizer.save_pretrained('./fine_tuned_rap_model')

In [ ]:
# metrics = trainer.evaluate(eval_dataset=tokenized_test_dataset)
# experiment.log_metrics(metrics)

In [ ]:
experiment.end()